In [0]:
imagesFolder = '/home/ImageNetPart/'
%tensorflow_version 2.x
# %rm Data/*.dat
!find /home -name *.dat

def initColab_Full():
    from google.colab import drive
    # !set | more
    drive.mount('/home/gdrive')
    %cd "/home/gdrive/My Drive/Visualiz_Zeiler"

    !pip uninstall imageio --yes
    !pip install imageio
    !pip uninstall keras --yes
    !pip install keras

    !df -hm | grep overlay
    !unzip VKIImageNetPart_TrainTestDivided.zip -d $imagesFolder >/dev/null
    !df -hm
    %mv "$imagesFolder"Test "$imagesFolder"test
    %mv "$imagesFolder"Train "$imagesFolder"train

    %ls -l "/home/ImageNetPart/test/Baltimore oriole/2543"*
    path = imagesFolder + 'train/baseball/2832539985_13d806a911'
    # %ls -l "$path"* 
    %mv "$path"".jpg" "$path"".png"
    %ls -l "$path"* 

    path = imagesFolder + 'test/Baltimore oriole/2543362638_8d77863b4c'
    %mv "$path"".jpg" "$path"".png"
    %ls -l "$path"* 
    
def initCurRuntime():
    %cd "/home/gdrive/My Drive/Visualiz_Zeiler"
    # %rm *\(1\)*
    # %ls *Lo* -l
    # %pwd
    # %rm *.h5
    # %pip freeze >req.txt


    import matplotlib.pyplot as plt
    import matplotlib
    from matplotlib.pyplot import imshow   # Doesn't work without this for some reason
    %matplotlib inline
    %mkdir -p "Logs2/src"
    %cp -f ./*py* Logs2/src/

    %mkdir QtLogs

initColab_Full()    # Mounts Google drive and unzips images
initCurRuntime()

%load_ext autoreload
%autoreload 2
from VisColabUtils import *
from VisUtils import *

controlObj = VisJupyterNotUtils.NetControlObject()
controlObj.init()

# path = '2/10'
# !echo "echo  $path""*"
# %mv "$path"*.png "$path"".pn_g"
# # !mv "$path""*.jpg" "$path"".png"
# %ls "$path"*

In [0]:
# print(controlObj.netWrapper.imageDataset.testSubset._loadFilesTree())
# print(controlObj.netWrapper.imageDataset.trainSubset._loadFilesTree())
# for im in controlObj.netWrapper.imageDataset.testSubset.imagesFileNames:
#     if im.find('2543') >= 0:
#         print(im)

# !pip uninstall matplotlib python-qt imgaug
# !python -m pip install --upgrade pip
# !pip install matplotlib
# !pip install python-qt4

In [0]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
@interact   # Doesn't work now at VKI's GeForce, works at CoLab
def f(x=5):
    image = controlObj.imageDataset.getImage(x, 'cropped')
    imshow(image);
    return x
# interact(f, x=10);

imageNum = 8
# image = controlObj.imageDataset.getImage(imageNum, 'cropped')
# imshow(image);

In [0]:
epochNum = -3
print(tf.__version__)
print(controlObj.getSelectedEpochNum())
colCount = 10
layerName = 'conv_11'
margin = 3
activations, drawMode, stdData = controlObj.getActivationsData(epochNum, imageNum, layerName)
actImage = layoutLayersToOneImage(np.sqrt(activations), colCount, margin)
print(activations.shape, actImage.shape)
plt.imshow(actImage);

In [0]:
for i in range(0):
    activations = controlObj.netWrapper.getImageActivations('dense_2', i, epochNum)
    print(activations);

In [0]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range
  return image
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)
def load_and_preprocess_image_with_label(path, label):
  print(path)
  image = tf.io.read_file(path)
  print(image, label)
  return preprocess_image(image), label

all_image_paths = ["ImageNetPart/American crow/1003355_8d39d66686.jpg", 
                   "ImageNetPart/barn owl/101715431_19d0b37108.jpg"]
labels = ['crow', 'owl']
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=1)
# for n, image in enumerate(load_image_ds.take(7)):
#     print(image)
    
label_ds = tf.data.Dataset.from_tensor_slices(labels)
ds = tf.data.Dataset.zip((image_ds, label_ds))
# ds = tf.data.Dataset.zip((path_ds, label_ds))
# for n, v in enumerate(ds.take(7)):
#     print(v)
ds = ds.repeat()
# ds = ds.map(load_and_preprocess_image_with_label, num_parallel_calls=1)

In [0]:
# Internal dataset's check
import tensorflow as tf
print(tf.executing_eagerly())
ds = controlObj.imageDataset.getTfDataset('test') 
# print(ds)
plt.figure(figsize=(8,8))
for n, val in enumerate(ds.take(3)):
    # ??val
    # print(val)
    imageNum = val[0].numpy()
    # classInd = val[1].numpy()
    classInd = val[1].numpy()
#     if imageNum % 1000 == 0:
#         print(val)
#     continue
    className = controlObj.imageDataset.getClassNameLabel(classInd) #.decode('ascii')
#     print(image.numpy().shape)
    ax = plt.subplot(4,4,n+1)
    plt.text(1.1, 0.5, '%s (%s)' % (className, str(classInd)),  
          transform = ax.transAxes)
    image = controlObj.imageDataset.getImage(imageNum, 'cropped', 'test')
    plt.imshow(image / 255.0)
    #   plt.legend( bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [0]:
# Final (batched) datasets' check
# controlObj.netWrapper._initMainNet()
# print(controlObj.netWrapper)
trainImageNums = np.arange(1, controlObj.imageDataset.getImageCount('train') + 1)
testImageNums = np.arange(1, controlObj.imageDataset.getImageCount('test') + 1)
tfTrainDataset, tfTestDataset = controlObj.netWrapper.net._getTfDatasets(
                trainImageNums, testImageNums, 1000)
# tfDataset = tf.compat.v1.data.make_one_shot_iterator(tfDataset)
# ??tfDataset

print(controlObj.imageDataset.getImageCount('train'))
if 1:
    plt.figure(figsize=(8,8))
    for n, val in enumerate(tfTrainDataset.take(2)):
        print(controlObj.netWrapper.getCacheStatusInfo(True))
        images = val[0].numpy()
    #     ??images
        print(images.shape)
        matrixImage = layoutLayersToOneImage(images, colCount, margin)
        imshow(matrixImage);
    #     imshow(images[0])
        plt.show()

if 0:
 for n, val in enumerate(tfDataset.take(0)):
  # ??val
  image = val[0][0].numpy()
  # classInd = val[1].numpy()
  # className = controlObj.imageDataset.getClassNameByInd(classInd) #.decode('ascii')
  classVec = val[1][0].numpy()
  print(image.shape)
  ax = plt.subplot(4,4,n+1)
  plt.text(1.1, 0.5, str(classVec),  
          transform = ax.transAxes)
  plt.imshow(image /255.0)
#   plt.legend( bbox_to_anchor=(1.05, 1), loc='upper left')
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.show()


In [0]:
controlObj.netWrapper.imageCache.maxMemory = 4 << 30
controlObj.learnRate = 7e-5
controlObj.onDoItersPressed(1)
print(controlObj.netWrapper.getCacheStatusInfo(True))
controlObj.onDoItersPressed(20000)

In [0]:
controlObj.learnRate = 7e-6
controlObj.onDoItersPressed(50000)

In [0]:
controlObj.learnRate = 7e-7
controlObj.onDoItersPressed(1000000)

In [0]:
plt.figure(figsize=(20,10))
columns = 5
for i in range(3):
    imshow(activations[i],cmap='Greys_r')   # Displays only one
#display(activations[1])

In [0]:
!nvidia-smi

In [0]:
def showImagesMatrix(data, col=2):
    fig = figure( figsize=(600, 300))
    number_of_files = len(data)
    print(number_of_files)
    row = number_of_files/col
    if (number_of_files%col != 0):
        row += 1
    for i in range(number_of_files):
        a=fig.add_subplot(row,col,i+1)
        # image = imread(mypath+'/'+list_of_files[i])
        axis('off')
        print('Image %d drawn' % i)
    # Very slow at the end
        
showImagesMatrix(activations[1:8])

In [0]:
for i in np.random.permutation(600)[:4]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)

In [0]:
im = Image(filename="image/000644.jpg", width=100, height=100)
im.shape()

In [0]:
![title](images/000644.png)

In [0]:
from IPython.display import Image          # Displaying of multiple images. To check
from IPython.display import display
x = Image(filename='1.png') 
y = Image(filename='2.png') 
display(x, y)

In [0]:
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    display(widgets.IntSlider())